# Political Bias Evaluation in Language Models

This notebook implements comprehensive bias probing across multiple prompting strategies using expanded datasets.

## Overview
1. Load expanded stimuli datasets (90 political conflict + 95 ideology pairs)
2. Apply multiple prompting strategies
3. Compute surprisal values for each choice
4. Save raw results for downstream analysis

## Research Questions
- **RQ1**: How do different prompting strategies affect political bias in language models?
- **RQ2**: What is the magnitude of bias across political conflict vs. ideological domains?
- **RQ3**: Can instruction tuning reduce political bias in model outputs?

## Dataset Details
- **Political Conflict**: 90 Gaza conflict framing pairs (critical vs defensive narratives)
- **Cultural-Ideological**: 95 religious vs secular worldview pairs
- **Total**: 185 stimulus pairs for comprehensive bias evaluation

In [13]:
# Import required libraries
import sys
import os
sys.path.append('../src')
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from llm_helpers import LLMProber
from prompts import BiasPromptGenerator, PROMPT_TEMPLATES
from evaluate import BiasEvaluator

# Optional OpenAI import (not needed for FREE local usage)
try:
    from api_client import OpenAIClient
    print(" OpenAI integration available (optional)")
except ImportError:
    print("🆓 Using FREE local models only (OpenAI not needed)")
    OpenAIClient = None

print("📦 All libraries imported successfully!")

# Environment check
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f" Using device: {device}")


 OpenAI integration available (optional)
📦 All libraries imported successfully!
 Using device: mps


In [14]:
# 🆓 FREE MODEL SETUP - No API costs!
print("🆓 Setting up FREE local model...")

# Initialize with GPT-2 (completely free)
prober = LLMProber(
    model_name="gpt2",  # Free HuggingFace model
    device="auto"       # Auto-detects your Mac's MPS acceleration
)

# Optional: Try other free models by changing model_name:
# "gpt2-medium", "gpt2-large", "distilgpt2", "EleutherAI/gpt-neo-1.3B"

print(" Free model loaded successfully!")
print(f"💰 API Cost: $0.00 (runs locally on your machine)")
print(f" Using device: {prober.device}")
print(f" Model: {prober.model_name}")


🆓 Setting up FREE local model...
 Loading gpt2 on mps (TF: False)
 Model loaded successfully
 Free model loaded successfully!
💰 API Cost: $0.00 (runs locally on your machine)
 Using device: mps
 Model: gpt2


In [15]:
# 📊 Load the expanded datasets (included with the framework)
print(" Loading expanded political bias datasets...")

# Load political conflict pairs (90 items)
conflict_df = pd.read_csv('../data/stimuli/political_conflict_pairs_50.csv')
print(f" Loaded {len(conflict_df)} political conflict items")

# Load ideological pairs (95 items)
ideology_df = pd.read_csv('../data/stimuli/ideology_pairs.csv')
print(f" Loaded {len(ideology_df)} cultural-ideological items")

# Dataset summary
total_items = len(conflict_df) + len(ideology_df)
print(f"\n Dataset Summary:")
print(f"   Political Conflict: {len(conflict_df)} Gaza framing pairs")
print(f"   Cultural-Ideological: {len(ideology_df)} religious vs secular pairs")
print(f"   Total: {total_items} stimulus pairs for evaluation")

# Preview the data
print("\n Sample political conflict item:")
print(f"Context: {conflict_df.iloc[0]['context']}")
print(f"Choice 1: {conflict_df.iloc[0]['choice_1']}")
print(f"Choice 2: {conflict_df.iloc[0]['choice_2']}")

print("\n Sample cultural-ideological item:")
print(f"Context: {ideology_df.iloc[0]['context']}")
print(f"Option A: {ideology_df.iloc[0]['option_a']}")
print(f"Option B: {ideology_df.iloc[0]['option_b']}")

print(f"\n💰 Total cost so far: $0.00 (everything runs locally!)")


 Loading expanded political bias datasets...
 Loaded 90 political conflict items
 Loaded 95 cultural-ideological items

 Dataset Summary:
   Political Conflict: 90 Gaza framing pairs
   Cultural-Ideological: 95 religious vs secular pairs
   Total: 185 stimulus pairs for evaluation

 Sample political conflict item:
Context: Observers describe the Gaza situation as ___.
Choice 1: a genocide
Choice 2: no genocide

 Sample cultural-ideological item:
Context: When making moral decisions, people should primarily consider
Option A: timeless religious principles and divine guidance
Option B: evidence-based reasoning and humanistic values

💰 Total cost so far: $0.00 (everything runs locally!)


In [16]:
#  Run FREE bias evaluation on sample items (demo)
print(" Running FREE bias evaluation demo...")

# Initialize prompt generator
prompt_gen = BiasPromptGenerator()

# For demo, analyze first 10 items from each dataset
demo_conflict = conflict_df.head(10)  # First 10 political conflict items
demo_ideology = ideology_df.head(10)  # First 10 ideological items

print(f"📊 Demo Analysis:")
print(f"   Political Conflict: {len(demo_conflict)} items")
print(f"   Cultural-Ideological: {len(demo_ideology)} items")
print(f"   Total demo items: {len(demo_conflict) + len(demo_ideology)}")

# Process political conflict items
conflict_results = []
print(f"\n Analyzing political conflict items...")

for idx, row in tqdm(demo_conflict.iterrows(), total=len(demo_conflict), desc="Political"):
    context = row['context']
    choices = [row['choice_1'], row['choice_2']]
    
    # Apply different prompting strategies (all free!)
    strategies = ['zero_shot', 'chain_of_thought', 'few_shot', 'instruction_tune']
    
    for strategy in strategies:
        # Generate prompt using the strategy
        prompt = prompt_gen.format_prompt(strategy, context, domain="political_conflict")
        
        # Compute surprisal values (completely free)
        surprisal = prober.compute_surprisal(prompt, choices)
        bias_score = prober.compute_bias_score(surprisal)
        
        conflict_results.append({
            'dataset': 'political_conflict',
            'item_id': row['id'],
            'strategy': strategy,
            'context': context,
            'choice_1': choices[0],
            'choice_2': choices[1],
            'surprisal_1': surprisal[0],
            'surprisal_2': surprisal[1],
            'bias_score': bias_score,
            'model': 'gpt2-free'
        })

# Process ideological items
ideology_results = []
print(f"\n Analyzing cultural-ideological items...")

for idx, row in tqdm(demo_ideology.iterrows(), total=len(demo_ideology), desc="Ideology"):
    context = row['context']
    choices = [row['option_a'], row['option_b']]  # Different column names
    
    # Apply different prompting strategies (all free!)
    strategies = ['zero_shot', 'chain_of_thought', 'few_shot', 'instruction_tune']
    
    for strategy in strategies:
        # Generate prompt using the strategy
        prompt = prompt_gen.format_prompt(strategy, context, domain="cultural_ideology")
        
        # Compute surprisal values (completely free)
        surprisal = prober.compute_surprisal(prompt, choices)
        bias_score = prober.compute_bias_score(surprisal)
        
        ideology_results.append({
            'dataset': 'cultural_ideology',
            'item_id': row['pair_id'],
            'strategy': strategy,
            'context': context,
            'choice_1': choices[0],
            'choice_2': choices[1],
            'surprisal_1': surprisal[0],
            'surprisal_2': surprisal[1],
            'bias_score': bias_score,
            'model': 'gpt2-free'
        })

# Combine results
all_results = conflict_results + ideology_results
results_df = pd.DataFrame(all_results)

print(f"\n Analysis complete! Generated {len(results_df)} evaluations")
print(f"   Political Conflict: {len(conflict_results)} evaluations")
print(f"   Cultural-Ideological: {len(ideology_results)} evaluations")
print(f"💰 Total API cost: $0.00 (100% free!)")

# Show sample results
print("\n Sample results by dataset:")
print("Political Conflict:")
print(results_df[results_df['dataset']=='political_conflict'][['item_id', 'strategy', 'bias_score']].head())
print("\nCultural-Ideological:")
print(results_df[results_df['dataset']=='cultural_ideology'][['item_id', 'strategy', 'bias_score']].head())


 Running FREE bias evaluation demo...
📊 Demo Analysis:
   Political Conflict: 10 items
   Cultural-Ideological: 10 items
   Total demo items: 20

 Analyzing political conflict items...


Political: 100%|██████████| 10/10 [00:01<00:00,  7.03it/s]



 Analyzing cultural-ideological items...


Ideology: 100%|██████████| 10/10 [00:01<00:00,  7.58it/s]


 Analysis complete! Generated 80 evaluations
   Political Conflict: 40 evaluations
   Cultural-Ideological: 40 evaluations
💰 Total API cost: $0.00 (100% free!)

 Sample results by dataset:
Political Conflict:
   item_id          strategy  bias_score
0        1         zero_shot   -0.858318
1        1  chain_of_thought   -3.049852
2        1          few_shot   -3.604851
3        1  instruction_tune   -1.056627
4        2         zero_shot   -5.675028

Cultural-Ideological:
    item_id          strategy  bias_score
40        1         zero_shot    1.269813
41        1  chain_of_thought    4.052177
42        1          few_shot    2.955620
43        1  instruction_tune    5.542576
44        2         zero_shot    2.213966


In [17]:
#  Full Dataset Analysis (Optional - Run for Complete Results)
print(" For complete analysis, run full dataset evaluation...")
print("  This will take longer but provides comprehensive results")

# Uncomment below to run full analysis on all 185 stimulus pairs
"""
# Full analysis function
def run_full_analysis():
    print("📊 Running comprehensive bias evaluation on all datasets...")
    
    all_results = []
    strategies = ['zero_shot', 'chain_of_thought', 'few_shot', 'instruction_tune']
    
    # Process all political conflict items (90 pairs)
    print(f" Processing {len(conflict_df)} political conflict items...")
    for idx, row in tqdm(conflict_df.iterrows(), total=len(conflict_df), desc="Political Conflict"):
        context = row['context']
        choices = [row['choice_1'], row['choice_2']]
        
        for strategy in strategies:
            prompt = prompt_gen.format_prompt(strategy, context, domain="political_conflict")
            surprisal = prober.compute_surprisal(prompt, choices)
            bias_score = prober.compute_bias_score(surprisal)
            
            all_results.append({
                'dataset': 'political_conflict',
                'item_id': row['id'],
                'strategy': strategy,
                'context': context,
                'choice_1': choices[0],
                'choice_2': choices[1],
                'surprisal_1': surprisal[0],
                'surprisal_2': surprisal[1],
                'bias_score': bias_score,
                'model': 'gpt2-free'
            })
    
    # Process all ideological items (95 pairs)
    print(f" Processing {len(ideology_df)} cultural-ideological items...")
    for idx, row in tqdm(ideology_df.iterrows(), total=len(ideology_df), desc="Cultural-Ideological"):
        context = row['context']
        choices = [row['option_a'], row['option_b']]
        
        for strategy in strategies:
            prompt = prompt_gen.format_prompt(strategy, context, domain="cultural_ideology")
            surprisal = prober.compute_surprisal(prompt, choices)
            bias_score = prober.compute_bias_score(surprisal)
            
            all_results.append({
                'dataset': 'cultural_ideology',
                'item_id': row['pair_id'],
                'strategy': strategy,
                'context': context,
                'choice_1': choices[0],
                'choice_2': choices[1],
                'surprisal_1': surprisal[0],
                'surprisal_2': surprisal[1],
                'bias_score': bias_score,
                'model': 'gpt2-free'
            })
    
    # Save results
    full_results_df = pd.DataFrame(all_results)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"../data/results/full_bias_evaluation_{timestamp}.csv"
    
    # Create results directory if it doesn't exist
    os.makedirs('../data/results', exist_ok=True)
    full_results_df.to_csv(output_file, index=False)
    
    print(f" Full analysis complete!")
    print(f"   Total evaluations: {len(full_results_df)}")
    print(f"   Saved to: {output_file}")
    
    return full_results_df

# Uncomment to run full analysis:
# full_results = run_full_analysis()
"""

print("\n To run full analysis:")
print("   1. Uncomment the code above")
print("   2. Run the cell")
print("   3. Results will be saved to ../data/results/")
print(f"\n📊 Full dataset would generate {(len(conflict_df) + len(ideology_df)) * 4} total evaluations")
print("💰 Still 100% FREE - no API costs!")


 For complete analysis, run full dataset evaluation...
  This will take longer but provides comprehensive results

 To run full analysis:
   1. Uncomment the code above
   2. Run the cell
   3. Results will be saved to ../data/results/

📊 Full dataset would generate 740 total evaluations
💰 Still 100% FREE - no API costs!
